In [3]:
import io
import os
import multiprocessing
import requests

import pandas as pd
import tqdm
from bs4 import BeautifulSoup
from pypdf import PdfReader

In [15]:
def _fetch_page_content(page_num=1):
  url = f"https://www.b3.com.br/pt_br/regulacao/oficios-e-comunicados/?pagination={page_num}"

  # ToDo - retry on except
  try:
    response = requests.get(url)
  except Exception as e:
    raise e

  return BeautifulSoup(response.text, 'html.parser')

In [16]:
def _get_pdf_content(url):
  
  # ToDo - retry on except
  return requests.get(url)

In [42]:
from time import sleep
from random import randint

In [53]:
def scrape_b3(page_num):
    if page_num % 5 == 0: 
        sleep(randint(0,10))
        
    save_path = "/home/gabriel/Documentos/projects/poli-capstone-project/data/pdf/"
    soup = _fetch_page_content(page_num)
    content_divs = soup.select('li.accordion-navigation')
    
    base_url = "https://www.b3.com.br"
    data = []

    # For each published communication
    for content in content_divs:
        published_date = content.select("div.least-content")[0].text
        published_title = content.select("div.content p.primary-text")[0].text
        published_abstract = content.select("div.content p.resumo-oficio")[0].text
        published_subject = content.select("div.content p.assunto-oficio")[0].text
        communication_link = content.select("div.content ul li a")[0].get("href", None)
        
        url = base_url + communication_link
        
        pdf_response = _get_pdf_content(url)
        pdf_bytes = pdf_response.content

        file_date = published_date.replace("/", "_")
        file_title = published_title.replace("/", "_")
        file_name = f'{save_path}{file_date}|{file_title}.pdf'

        data.append([published_date, published_title, published_abstract, published_subject, url, file_name])

        with open(file_name, 'wb') as f:
            f.write(pdf_bytes)

        return pd.DataFrame(data, columns=["published_date", "published_title", "published_abstract", "published_subject", "url", "file_name"])

In [54]:
num_processes = 4
pagination = [i for i in range(1, 285)] # There are 284 pages as of 2023-10-29

with multiprocessing.Pool(processes=num_processes) as pool:
    results = list(
        tqdm.tqdm(
        pool.imap(scrape_b3, pagination),
        total=len(pagination)
        )
    )

100%|████████████████████████████████████████████████████████████████| 284/284 [04:50<00:00,  1.02s/it]


In [63]:
save_path = "/home/gabriel/Documentos/projects/poli-capstone-project/data/pdf/"
df = pd.concat(results)

df_replaced = df.assign(
    file_name=df["file_name"].str.replace(save_path, "")
)
df_replaced.head()

,published_date,published_title,published_abstract,published_subject,url,file_name
0,26/10/23,174-2023-PRE-Ofício Circular,"Para o presente programa, serão credenciados d...",Processo para Credenciamento no Programa de Fo...,https://www.b3.com.br/data/files/11/52/56/89/C...,26_10_23|174-2023-PRE-Ofício Circular.pdf
0,24/10/23,099-2023-VNC-Comunicado Externo,"A B3 informa que, a partir do dia 30/10/2023, ...",Novos grupos de negociação para operações estr...,https://www.b3.com.br/data/files/AA/D4/F5/61/4...,24_10_23|099-2023-VNC-Comunicado Externo.pdf
0,10/10/23,164-2023-PRE-Ofício Circular,"A B3 informa que, em 16/10/2023, às 15h, será ...",Tratamento das Carteiras de Índices da B3 em v...,https://www.b3.com.br/data/files/58/50/FF/7B/2...,10_10_23|164-2023-PRE-Ofício Circular.pdf
0,05/10/23,051-2023-VPC-Comunicado Externo,"Informamos que, a partir de 15/01/2024, as tab...",Reajuste de Preços dos Serviços de Conectivida...,https://www.b3.com.br/data/files/E4/92/39/AB/6...,05_10_23|051-2023-VPC-Comunicado Externo.pdf
0,28/09/23,090-2023-VNC-Comunicado Externo,A B3 informa que está disponível para download...,Gateway Binário – Atualização do Template,https://www.b3.com.br/data/files/3B/61/7B/62/9...,28_09_23|090-2023-VNC-Comunicado Externo.pdf


In [65]:
df_replaced.to_parquet("../data/list_b3.parquet", engine="pyarrow")

In [89]:
def extract_text_with_pypdf(dataframe_row):
    save_path = "/home/gabriel/Documentos/projects/poli-capstone-project/data/pdf/"
    file_name = dataframe_row
    
    with open(f'{save_path}{file_name}', "rb") as pdf_file:
        reader = PdfReader(pdf_file)
        pages = reader.pages
        
        full_text = ""
        for page in pages:
          full_text = full_text + page.extract_text()

    return full_text

In [91]:
df_replaced['pypdf_extraction'] = df_replaced["file_name"].apply(extract_text_with_pypdf)
df_replaced.tail()

,published_date,published_title,published_abstract,published_subject,url,file_name,pypdf_extraction
0,10/05/17,034-2017-DP-Ofício Circular,"O tratamento, pelo Serviço de Empréstimo de At...",Tratamento das Posições de Empréstimo de Ativo...,https://www.b3.com.br/data/files/42/A7/73/6D/9...,10_05_17|034-2017-DP-Ofício Circular.pdf,\n \n 1 \n10 de maio de 201 7 \n034/2017 ...
0,13/04/17,031-2017-DP-Ofício Circular,A B3 informa o início do Processo para Credenc...,Processo para Credenciamento de Formador de Me...,https://www.b3.com.br/data/files/69/47/DB/65/0...,13_04_17|031-2017-DP-Ofício Circular.pdf,\n \n 1 \n13 de abril de 201 7 \n031/2017...
0,29/03/17,027-2017-DP-Ofício Circular,"A Bolsa implementará, em sua plataforma PUMA T...",Plataforma PUMA Trading System B3 – Implementa...,https://www.b3.com.br/data/files/B9/91/AD/B1/9...,29_03_17|027-2017-DP-Ofício Circular.pdf,\n 1 \n \n \n \n \n \n29 de março de 201...
0,23/03/17,025-2017-DO-Comunicado Externo,"Em 24/03/2017, o Banco Itaú BBA S.A. realizará...",Distribuição Pública Primária e Secundária de ...,https://www.b3.com.br/data/files/7D/33/FA/A1/4...,23_03_17|025-2017-DO-Comunicado Externo.pdf,\n \n23 de março de 2017 \n025/2017 -DO \nC...
0,31/05/12,081-2012DO-Comunicado Externo,"A BM&FBOVESPA oferecerá, a partir de 04/06/201...",Transferência de Proventos Provisionados e Man...,https://www.b3.com.br/data/files/B1/B4/76/24/F...,31_05_12|081-2012DO-Comunicado Externo.pdf,


In [100]:
# entries that are empty
df_replaced[df_replaced["pypdf_extraction"].apply(len) <= 5]

,published_date,published_title,published_abstract,published_subject,url,file_name,pypdf_extraction
0,18/04/19,004-2019-VPC-Ofício Circular,Informamos o lançamento da Política Comercial ...,Market Data B3 – Nova Política Comercial de Di...,https://www.b3.com.br/data/files/2E/D4/94/79/5...,18_04_19|004-2019-VPC-Ofício Circular.pdf,
0,31/05/12,081-2012DO-Comunicado Externo,"A BM&FBOVESPA oferecerá, a partir de 04/06/201...",Transferência de Proventos Provisionados e Man...,https://www.b3.com.br/data/files/B1/B4/76/24/F...,31_05_12|081-2012DO-Comunicado Externo.pdf,


In [101]:
df_replaced.to_parquet("../data/list_b3_pdf_comparison.parquet", engine="pyarrow")

The pdfminer library will be tested. The <a href="https://github.com/euske/pdfminer">main</a> library appear to be dormant currently, and it's recommended in their github page to use <a href="https://github.com/pdfminer/pdfminer.six">pdfminer.six</a>. So, this version will be the tested one.
Also, based on recent benchmarks presented over this benchmarks <a href="https://github.com/py-pdf/benchmarks">repo</a> on Github, pdfium2 will also be tested, as it's the library that presents the best results over the benchmarks there.

In [107]:
from pdfminer.high_level import extract_text

def extract_text_with_pdfminer(dataframe_row):
    save_path = "/home/gabriel/Documentos/projects/poli-capstone-project/data/pdf/"
    file_name = dataframe_row
    
    full_text = extract_text(f'{save_path}{file_name}')
    return full_text

In [108]:
df_replaced['pdfminer_extraction'] = df_replaced["file_name"].apply(extract_text_with_pdfminer)
df_replaced.tail()

,published_date,published_title,published_abstract,published_subject,url,file_name,pypdf_extraction,pdfminer_extraction
0,10/05/17,034-2017-DP-Ofício Circular,"O tratamento, pelo Serviço de Empréstimo de At...",Tratamento das Posições de Empréstimo de Ativo...,https://www.b3.com.br/data/files/42/A7/73/6D/9...,10_05_17|034-2017-DP-Ofício Circular.pdf,\n \n 1 \n10 de maio de 201 7 \n034/2017 ...,10 de maio de 2017 \n\n034/2017-DP \n\nO F Í C...
0,13/04/17,031-2017-DP-Ofício Circular,A B3 informa o início do Processo para Credenc...,Processo para Credenciamento de Formador de Me...,https://www.b3.com.br/data/files/69/47/DB/65/0...,13_04_17|031-2017-DP-Ofício Circular.pdf,\n \n 1 \n13 de abril de 201 7 \n031/2017...,13 de abril de 2017 \n\n031/2017-DP \n\nO F Í ...
0,29/03/17,027-2017-DP-Ofício Circular,"A Bolsa implementará, em sua plataforma PUMA T...",Plataforma PUMA Trading System B3 – Implementa...,https://www.b3.com.br/data/files/B9/91/AD/B1/9...,29_03_17|027-2017-DP-Ofício Circular.pdf,\n 1 \n \n \n \n \n \n29 de março de 201...,29 de março de 2017 \n\n027/2017-DP \n\nO F Í ...
0,23/03/17,025-2017-DO-Comunicado Externo,"Em 24/03/2017, o Banco Itaú BBA S.A. realizará...",Distribuição Pública Primária e Secundária de ...,https://www.b3.com.br/data/files/7D/33/FA/A1/4...,23_03_17|025-2017-DO-Comunicado Externo.pdf,\n \n23 de março de 2017 \n025/2017 -DO \nC...,23 de março de 2017 \n025/2017-DO \n\nC O M U ...
0,31/05/12,081-2012DO-Comunicado Externo,"A BM&FBOVESPA oferecerá, a partir de 04/06/201...",Transferência de Proventos Provisionados e Man...,https://www.b3.com.br/data/files/B1/B4/76/24/F...,31_05_12|081-2012DO-Comunicado Externo.pdf,,


In [109]:
# entries that are empty: same between the different libraries
df_replaced[df_replaced["pdfminer_extraction"].apply(len) <= 5]

,published_date,published_title,published_abstract,published_subject,url,file_name,pypdf_extraction,pdfminer_extraction
0,18/04/19,004-2019-VPC-Ofício Circular,Informamos o lançamento da Política Comercial ...,Market Data B3 – Nova Política Comercial de Di...,https://www.b3.com.br/data/files/2E/D4/94/79/5...,18_04_19|004-2019-VPC-Ofício Circular.pdf,,
0,31/05/12,081-2012DO-Comunicado Externo,"A BM&FBOVESPA oferecerá, a partir de 04/06/201...",Transferência de Proventos Provisionados e Man...,https://www.b3.com.br/data/files/B1/B4/76/24/F...,31_05_12|081-2012DO-Comunicado Externo.pdf,,
